<a href="https://colab.research.google.com/github/marianneo812/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (137 kB/s)
Reading package lists... Done


In [17]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [18]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [19]:
# Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

# Example query: Selecting data from the temporary view
# Correct the column name to 'price'
result_df = spark.sql("SELECT * FROM home_sales WHERE price > 500000")

# Show the results of the query
result_df.show()


+--------------------+----------+----------+-------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+-------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016| 936923|       4|        3|       3167|   11733|     2|         1|  76|
|15a5830f-1529-4c5...|2019-07-06|      2010| 843191|       3|        6|       5086|   14637|     2|         0|  91|
|fd8213d0-3872-4de...|2020-04-23|      2017| 858900|       4|        5|       4465|    9873|     1|         1|  53|
|44b0b335-2cbd-433...|2020-06-04|      2013|1467672|       7|        3|       5510|    8269|     1|         1|  93|
|3d5545f8-bd3b-476...|2020-09-19|      2015| 797862|       4|        6|       3494|   10385|     2|         0|  90|
|c7d09d09-7d76-4b1...|2019-06-02|      2014| 618191|       4|        4| 

In [20]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

average_price_query = """
SELECT
    YEAR(date) as Year,
    ROUND(AVG(price), 2) as AveragePrice
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date)
ORDER BY
    Year
"""

# Execute the query
average_price_result = spark.sql(average_price_query)

# Show the results of the query
average_price_result.show()




+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [21]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# SQL query to find the average price of homes with 3 bedrooms and 3 bathrooms, grouped by year built
query = """
SELECT
    date_built AS YearBuilt,
    ROUND(AVG(price), 2) AS AveragePrice
FROM
    home_sales
WHERE
    bedrooms = '3' AND bathrooms = '3'
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
result_df = spark.sql(query)

# Show the results of the query
result_df.show()


+---------+------------+
|YearBuilt|AveragePrice|
+---------+------------+
|     2010|   292859.62|
|     2011|   291117.47|
|     2012|   293683.19|
|     2013|   295962.27|
|     2014|   290852.27|
|     2015|    288770.3|
|     2016|   290555.07|
|     2017|   292676.79|
+---------+------------+



In [22]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
SELECT
    date_built AS YearBuilt,
    ROUND(AVG(price), 2) AS AveragePrice
FROM
    home_sales
WHERE
    bedrooms = '3' AND
    bathrooms = '3' AND
    floors = '2' AND
    sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query
result_df = spark.sql(query)

# Show the results of the query
result_df.show()

+---------+------------+
|YearBuilt|AveragePrice|
+---------+------------+
|     2010|   285010.22|
|     2011|   276553.81|
|     2012|   307539.97|
|     2013|   303676.79|
|     2014|   298264.72|
|     2015|   297609.97|
|     2016|    293965.1|
|     2017|   280317.58|
+---------+------------+



In [23]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT
    view AS ViewRating,
    ROUND(AVG(price), 2) AS AveragePrice
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
result_df = spark.sql(query)

# Show the results of the query
result_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+------------+
|ViewRating|AveragePrice|
+----------+------------+
|        99|  1061201.42|
|        98|  1053739.33|
|        97|  1129040.15|
|        96|  1017815.92|
|        95|   1054325.6|
|        94|   1033536.2|
|        93|  1026006.06|
|        92|   970402.55|
|        91|  1137372.73|
|        90|  1062654.16|
|        89|  1107839.15|
|        88|  1031719.35|
|        87|   1072285.2|
|        86|  1070444.25|
|        85|  1056336.74|
|        84|  1117233.13|
|        83|  1033965.93|
|        82|   1063498.0|
|        81|  1053472.79|
|        80|   991767.38|
+----------+------------+
only showing top 20 rows

--- 2.0980305671691895 seconds ---


In [24]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [27]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
SELECT
    view AS ViewRating,
    ROUND(AVG(price), 2) AS AveragePrice
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
result_df = spark.sql(query)

# Show the results of the query
result_df.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+------------+
|ViewRating|AveragePrice|
+----------+------------+
|        99|  1061201.42|
|        98|  1053739.33|
|        97|  1129040.15|
|        96|  1017815.92|
|        95|   1054325.6|
|        94|   1033536.2|
|        93|  1026006.06|
|        92|   970402.55|
|        91|  1137372.73|
|        90|  1062654.16|
|        89|  1107839.15|
|        88|  1031719.35|
|        87|   1072285.2|
|        86|  1070444.25|
|        85|  1056336.74|
|        84|  1117233.13|
|        83|  1033965.93|
|        82|   1063498.0|
|        81|  1053472.79|
|        80|   991767.38|
+----------+------------+
only showing top 20 rows

--- 2.139293909072876 seconds ---


In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("home_sales_partitioned_by_date_built.parquet")

# Verify that the Parquet files are written and partitioned correctly
partitioned_df = spark.read.parquet("home_sales_partitioned_by_date_built.parquet")
partitioned_df.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [29]:
# 11. Read the parquet formatted data.
parquet_file_path = "home_sales_partitioned_by_date_built.parquet"

# Read the Parquet formatted data
partitioned_df = spark.read.parquet(parquet_file_path)

# Show the schema to verify the data structure
partitioned_df.printSchema()

# Show the first few rows of the DataFrame to verify the data
partitioned_df.show()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3| 

In [30]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("home_sales_parquet")

# Verify the creation of the temporary view by running a simple query
result_df = spark.sql("SELECT * FROM home_sales_parquet LIMIT 10")
result_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [31]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT
    view AS ViewRating,
    ROUND(AVG(price), 2) AS AveragePrice
FROM
    home_sales_parquet
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query
result_df = spark.sql(query)

# Show the results of the query
result_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+------------+
|ViewRating|AveragePrice|
+----------+------------+
|        99|  1061201.42|
|        98|  1053739.33|
|        97|  1129040.15|
|        96|  1017815.92|
|        95|   1054325.6|
|        94|   1033536.2|
|        93|  1026006.06|
|        92|   970402.55|
|        91|  1137372.73|
|        90|  1062654.16|
|        89|  1107839.15|
|        88|  1031719.35|
|        87|   1072285.2|
|        86|  1070444.25|
|        85|  1056336.74|
|        84|  1117233.13|
|        83|  1033965.93|
|        82|   1063498.0|
|        81|  1053472.79|
|        80|   991767.38|
+----------+------------+
only showing top 20 rows

--- 0.7645359039306641 seconds ---


In [32]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [33]:
# 15. Check if the home_sales is no longer cached

if not spark.catalog.isCached("home_sales"):
    print("The home_sales table has been successfully uncached.")
else:
    print("The home_sales table is still cached.")

The home_sales table has been successfully uncached.
